In [1]:
import scipy.io
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from pretrain_gnns.bio.model import GNN
from torch.cuda.amp import GradScaler, autocast
from torch.utils.checkpoint import checkpoint
from tqdm import tqdm

In [2]:
# Check if CUDA is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

scaler = GradScaler()

cuda


In [3]:
acm_data = scipy.io.loadmat('acmv9.mat')
citation_data = scipy.io.loadmat('citationv1.mat')

In [4]:
# Directly use sparse matrices for features and adjacency
X_train = torch.tensor(acm_data['attrb'].todense(), dtype=torch.float32).to(device)
y_train = torch.tensor(acm_data['group'].argmax(axis=1).squeeze(), dtype=torch.long).to(device)

X_test = torch.tensor(citation_data['attrb'].todense(), dtype=torch.float32).to(device)
y_test = torch.tensor(citation_data['group'].argmax(axis=1).squeeze(), dtype=torch.long).to(device)


In [5]:
# Prepare graph data with correct shape
edge_index_train = torch.tensor(acm_data['network'].nonzero(), dtype=torch.long).to(device).t().contiguous()
edge_index_test = torch.tensor(citation_data['network'].nonzero(), dtype=torch.long).to(device).t().contiguous()


C:\Users\akalps\AppData\Local\Temp\ipykernel_1164\408244624.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  edge_index_train = torch.tensor(acm_data['network'].nonzero(), dtype=torch.long).to(device).t().contiguous()


In [6]:
# Initialize edge_attr with the correct number of edges and dimensions (9 in this case)
edge_attr_train = torch.ones(edge_index_train.shape[1], 9).to(device)
edge_attr_test = torch.ones(edge_index_test.shape[1], 9).to(device)


In [7]:
# Create PyTorch Geometric data objects
train_data = Data(x=X_train, edge_index=edge_index_train, edge_attr=edge_attr_train).to(device)
test_data = Data(x=X_test, edge_index=edge_index_test, edge_attr=edge_attr_test).to(device)


In [8]:
# Use DataLoader for batching
train_loader = DataLoader([train_data], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data], batch_size=1, shuffle=False)

In [9]:
# Debug prints
print(f"X_train shape: {X_train.shape}")
print(f"edge_index_train shape: {edge_index_train.shape}")
print(f"edge_attr_train shape: {edge_attr_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"edge_index_test shape: {edge_index_test.shape}")
print(f"edge_attr_test shape: {edge_attr_test.shape}")

X_train shape: torch.Size([9360, 6775])
edge_index_train shape: torch.Size([31158, 2])
edge_attr_train shape: torch.Size([2, 9])
X_test shape: torch.Size([8935, 6775])
edge_index_test shape: torch.Size([30211, 2])
edge_attr_test shape: torch.Size([2, 9])


In [10]:
# Instantiate the GNN model from pretrain-gnns
model = GNN(num_layer=2, emb_dim=16, gnn_type='gin').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [11]:
def checkpointed_forward(*inputs):
    return checkpoint(model, *inputs)

In [12]:
# Training loop 

model.train()
for epoch in tqdm(range(25), desc="Training Progress"):
    for batch in train_loader:
        optimizer.zero_grad()
        with autocast():
            output = checkpointed_forward(batch.x, batch.edge_index, batch.edge_attr)
            loss = criterion(output, y_train)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        torch.cuda.empty_cache() 
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Training Progress:   0%|          | 0/25 [00:00<?, ?it/s]c:\Users\akalps\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\akalps\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Training Progress:   0%|          | 0/25 [00:01<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 31158 but got size 2 for tensor number 1 in the list.

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    for batch in test_loader:
        with autocast():
            output = checkpointed_forward(batch.x, batch.edge_index, batch.edge_attr)
        predictions = torch.argmax(output, dim=1)
        accuracy = accuracy_score(y_test.cpu(), predictions.cpu())  # Moving tensors back to CPU for metrics
        micro_f1 = f1_score(y_test.cpu(), predictions.cpu(), average='micro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Micro F1 Score: {micro_f1:.4f}')